In [2]:
# CELL 1: Model Download
import urllib.request
import os

# Define URLs
face_url = "https://storage.googleapis.com/mediapipe-models/face_landmarker/face_landmarker/float16/1/face_landmarker.task"
hand_url = "https://storage.googleapis.com/mediapipe-models/hand_landmarker/hand_landmarker/float16/1/hand_landmarker.task"

def download(url, name):
    if not os.path.exists(name):
        print(f"📥 Downloading {name}...")
        urllib.request.urlretrieve(url, name)
        print("✅ Done.")
    else:
        print(f"⚡ {name} ready.")

download(face_url, "face_landmarker.task")
download(hand_url, "hand_landmarker.task")

⚡ face_landmarker.task ready.
⚡ hand_landmarker.task ready.


In [3]:
# CELL 2: Critical Imports
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import cv2
import numpy as np
import time

print(f"MediaPipe Version: {mp.__version__}")
print("✅ Imports loaded. You can now run the video cells.")

MediaPipe Version: 0.10.31
✅ Imports loaded. You can now run the video cells.


In [4]:
# 1. SETUP
base_options = python.BaseOptions(model_asset_path='face_landmarker.task')
options = vision.FaceLandmarkerOptions(
    base_options=base_options,
    output_face_blendshapes=True,
    output_facial_transformation_matrixes=True,
    running_mode=vision.RunningMode.VIDEO, # <--- CRITICAL FIX
    num_faces=1)

detector = vision.FaceLandmarker.create_from_options(options)

In [5]:
# CELL 3: Face Mesh
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)

print("Press 'q' to stop.")

try:
    while True:
        ret, frame = cap.read()
        if not ret: break
        
        # 2. CONVERT
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        timestamp = int(time.time() * 1000)
        
        # 3. DETECT
        result = detector.detect_for_video(mp_image, timestamp)
        
        # 4. DRAW
        if result.face_landmarks:
            for landmarks in result.face_landmarks:
                for lm in landmarks:
                    x, y = int(lm.x * frame.shape[1]), int(lm.y * frame.shape[0])
                    cv2.circle(frame, (x, y), 1, (0, 255, 255), -1)

        cv2.imshow('Face Mesh', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'): break

except Exception as e:
    print(f"Error: {e}")
finally:
    cap.release()
    cv2.destroyAllWindows()
    detector.close() # cleanup AI memory

Press 'q' to stop.


In [8]:

# CELL 4: Hand Tracking
# 1. SETUP
base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
options = vision.HandLandmarkerOptions(
    base_options=base_options,
    num_hands=2,
    running_mode=vision.RunningMode.VIDEO, # <--- CRITICAL FIX
    min_hand_detection_confidence=0.5)

detector = vision.HandLandmarker.create_from_options(options)
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)

print("Press 'q' to stop.")
try:
    while True:
        ret, frame = cap.read()
        if not ret: break

        # 2. CONVERT
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        timestamp = int(time.time() * 1000)
        
        # 3. DETECT
        result = detector.detect_for_video(mp_image, timestamp)
        
        # 4. DRAW
        if result.hand_landmarks:
            for landmarks in result.hand_landmarks:
                for lm in landmarks:
                    x, y = int(lm.x * frame.shape[1]), int(lm.y * frame.shape[0])
                    cv2.circle(frame, (x, y), 3, (0, 255, 0), -1)

        cv2.imshow('Hand Tracking', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'): break

except Exception as e:
    print(f"Error: {e}")
finally:
    cap.release()
    cv2.destroyAllWindows()
    detector.close()

Press 'q' to stop.


In [9]:
# CELL 5: 3D Depth Analysis Chart
if 'detection_result' in locals() and detection_result.hand_landmarks:
    landmarks = detection_result.hand_landmarks[0]
    
    # Extract Z-coordinates (Depth relative to wrist)
    z_vals = [lm.z for lm in landmarks]
    indices = range(len(z_vals))
    
    plt.figure(figsize=(10, 5))
    plt.bar(indices, z_vals, color='purple')
    plt.title("Hand Depth Profile (How far is each finger?)")
    plt.xlabel("Landmark Index (0=Wrist, 8=Index Tip)")
    plt.ylabel("Relative Depth (Z-Axis)")
    plt.grid(True, alpha=0.3)
    plt.show()
else:
    print("Run Cell 4 first to capture some hand data!")

Run Cell 4 first to capture some hand data!
